In [ ]:
! pip install kaggle wget

In [ ]:
! nvidia-smi

In [ ]:
! mkdir ~ /.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download -d gverzea/edible-wild-plants
! unzip /content/edible-wild-plants.zip

In [ ]:
!git clone https://github.com/alexfrst/data-augmentation-benchmark

In [2]:
import sys

import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

sys.path.append("data-augmentation-benchmark")

import model.model as model
from model.evaluate import evaluate
from model.train import train_model
from utils.dataset import load_dataset
from utils.print_utils import Symbols


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(f"Working on {device} {Symbols.OK}")

train_loader, val_loader, test_loader, nb_classes = load_dataset("datasets/dataset", "datasets/dataset-test", batch_size=85)
inception_v3 = model.load_inception_for_tl(nb_classes)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", patience=5, factor=0.5, min_lr=1e-5)
criterion = nn.CrossEntropyLoss()

params_to_update = []
for name,param in inception_v3.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)

optimizer = optim.SGD(params_to_update, lr=0.001, momentum=0.9)

print("Apprentissage en transfer learning")
inception_v3.to(device)
inception_v3.train(True)
torch.manual_seed(42)

train_model(inception_v3, train_loader, val_loader, optimizer, criterion, evaluate, 65, device=device)
inception_v3.train(False)
loss, accuracy = evaluate(inception_v3, test_loader, device, criterion)
print("Accuracy (test): %.1f%%" % (100 * accuracy))

Working on cuda:0 ✓
Loading dataset...
Nombre d'images de train : 5902
Nombre d'images de val : 656
Nombre d'images de test : 310
Apprentissage sur 62 classes
Data loading and integrity check done ✓
Apprentissage en transfer learning


Epoch 0: 0it [00:00, ?it/s]

Inception3


Epoch 0: 1it [00:19, 19.04s/it, Acc (val)=0.0109, loss train=5.8, loss val=4.16]

Inception3


Epoch 0: 1it [00:30, 30.00s/it, Acc (val)=0.0109, loss train=5.8, loss val=4.16]


KeyboardInterrupt: 

Epoch 11: 70it [13:40, 11.72s/it, Acc (val)=0.447, loss train=2.6, loss val=2.36] 
Epoch 12: 70it [13:36, 11.66s/it, Acc (val)=0.462, loss train=3.3, loss val=2.32] 
Epoch 13: 70it [14:03, 12.05s/it, Acc (val)=0.453, loss train=2.89, loss val=2.29]
Epoch 14: 6it [01:12, 11.70s/it, Acc (val)=0.46, loss train=3.18, loss val=2.28] 